# Understanding Your FastAPI Backend (`main.py`)

This notebook breaks down the `main.py` script block by block to explain how our new data engine works. Each section explains a concept and then shows the corresponding code from the file.

## Block 1: Imports

This first block imports all the necessary libraries and modules that our application needs to function. Think of it as gathering your tools before you start building.

- **uvicorn**: The high-performance server that will run our API.
- **fastapi**: The main library for building the API itself.
- **sqlalchemy**: The library for interacting with our `mining_knowledge.db` database.
- **models**: This imports our `models.py` file so the API knows about our database table structure.

In [ ]:
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.staticfiles import StaticFiles
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker
from models import Base

## Block 2: Database Setup

This section establishes the connection to our SQLite database. It creates the `engine`, which is the core communication channel to the database file.

In [ ]:
DATABASE_URL = "sqlite:///./mining_knowledge.db"

engine = create_engine(
    DATABASE_URL, 
    connect_args={"check_same_thread": False} # A specific requirement for using SQLite with FastAPI
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

## Block 3: Create the FastAPI App

This single line creates the main `app` object. This object is the heart of our API; we will attach all our URLs (endpoints) to it.

In [ ]:
app = FastAPI()

## Block 4: API Endpoint to Get Table Names

This is our first "endpoint". The `@app.get("/api/tables")` line tells FastAPI that whenever a user goes to the URL `/api/tables`, the `get_table_names` function should run. This function uses SQLAlchemy's `inspect` tool to look inside the database and return a list of all the table names it finds.

In [ ]:
@app.get("/api/tables")
def get_table_names():
    """
    This endpoint inspects the database and returns a list of all table names.
    """
    inspector = inspect(engine)
    table_names = inspector.get_table_names()
    return table_names

## Block 5: API Endpoint to Get Table Data

This is a dynamic endpoint. The `{table_name}` in the URL is a variable. When the frontend requests `/api/table/practices`, FastAPI runs the `get_table_data` function and automatically passes the word `"practices"` into the `table_name` argument.

The function then:
1. Finds the correct Python model (e.g., the `Practice` class) that matches the table name.
2. Queries the database for all records from that table.
3. Converts the results into clean JSON and sends them back to the frontend.

In [ ]:
# Helper function to convert SQLAlchemy objects to dictionaries
def object_as_dict(obj):
    return {c.key: getattr(obj, c.key) for c in inspect(obj).mapper.column_attrs}

@app.get("/api/table/{table_name}")
def get_table_data(table_name: str):
    db = SessionLocal()
    try:
        ModelClass = None
        for mapper in Base.registry.mappers:
            if mapper.local_table.name == table_name:
                ModelClass = mapper.class_
                break
        
        if not ModelClass:
            raise HTTPException(status_code=404, detail="Table not found")

        records = db.query(ModelClass).all()
        result = [object_as_dict(rec) for rec in records]
        return result
    finally:
        db.close()

## Block 6: Serve the Frontend

This final piece of code tells our FastAPI application to also act as a simple web server. Any request that doesn't match one of our API endpoints (like `/api/tables`) will be treated as a request for a static file.

This is what allows a user to go to `http://localhost:8000` and get the `index.html` file, which then loads `style.css` and `script.js`. It's the replacement for the `python -m http.server` command.

In [ ]:
app.mount("/", StaticFiles(directory=".", html=True), name="static")